<a href="https://colab.research.google.com/github/reachusama/product-review-analysis/blob/main/Products_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Links

https://github.com/openai/openai-cookbook/blob/main/examples/How_to_handle_rate_limits.ipynb

https://www.sbert.net/examples/applications/clustering/README.html


Text -> Embeddings (Numerical Array) -> Lowered Dimensiont ot compress information -> visalisation/ML algos


In [ ]:
!pip install openai
!pip install pandas
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 847 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 17.4 MB/s 
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=1e9680be6f2ffd7b16184d23ab4bce2e2af969a385178538b2aa99ada2193f87
  Stored in directory: /root/.cache/pip/wheels/19/de/db/e82770b480ec30fd4a6d67108744b9c52be167c04fcf4af7b5
Successfully built openai
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import openai
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Usama Shahid

In [ ]:
df = pd.read_json("/content/drive/MyDrive/Product Reviews/Dataset/chunk_1.json")

## Dataset Exploration

In [ ]:
df.shape

(100000, 8)

In [ ]:
df.head()

,review_id,review_header,review_text,rating,review_posted_date,is_verified,reviews_count,brand
0,R4I8PJ9FR0GF3,FALSE ADVERTISING!!!!!! NOT AS ADVERISED! ALL ...,If I could give this a ZERO RATING I would!! I...,1.0,"November 17, 2020",1.0,107.0,Chef's Path
1,R1KHEH6D35PXM,"Weird, acrid chemical smell in the lids.",I was looking forward to using these container...,1.0,"September 25, 2020",1.0,107.0,Chef's Path
2,R35MGJ0HDCABLM,Buyer Beware- NOTHING like the picture,*BUYER BEWARE * NOTHING. LIKE. THE. PICTURE. I...,1.0,"January 10, 2021",1.0,107.0,Chef's Path
3,RK4HRLNJWC7MT,This is what I get for being busy,I have been busy mom and not able to shop in s...,1.0,"February 13, 2021",1.0,107.0,Chef's Path
4,RWGBX6LYL6KXK,TOO SMALL!!,I’m not quite sure why I thought these contain...,1.0,"November 16, 2020",1.0,107.0,Chef's Path


In [ ]:
df = df[["review_id", "review_header", "review_text", "rating", "review_posted_date", "is_verified", "reviews_count", "brand"]]

In [ ]:
df.head()

,review_id,review_header,review_text,rating,review_posted_date,is_verified,reviews_count,brand
0,R4I8PJ9FR0GF3,FALSE ADVERTISING!!!!!! NOT AS ADVERISED! ALL ...,If I could give this a ZERO RATING I would!! I...,1.0,"November 17, 2020",1.0,107.0,Chef's Path
1,R1KHEH6D35PXM,"Weird, acrid chemical smell in the lids.",I was looking forward to using these container...,1.0,"September 25, 2020",1.0,107.0,Chef's Path
2,R35MGJ0HDCABLM,Buyer Beware- NOTHING like the picture,*BUYER BEWARE * NOTHING. LIKE. THE. PICTURE. I...,1.0,"January 10, 2021",1.0,107.0,Chef's Path
3,RK4HRLNJWC7MT,This is what I get for being busy,I have been busy mom and not able to shop in s...,1.0,"February 13, 2021",1.0,107.0,Chef's Path
4,RWGBX6LYL6KXK,TOO SMALL!!,I’m not quite sure why I thought these contain...,1.0,"November 16, 2020",1.0,107.0,Chef's Path


In [ ]:
df.tail()

,review_id,review_header,review_text,rating,review_posted_date,is_verified,reviews_count,brand
99995,R2PVJ8KELJPCAO,7 piece outdoor sectional,Nice set,5.0,"October 24, 2020",1.0,11.0,Gotland
99996,R1AEYH2QOLCQHH,Awesome furniture,Great patio furniture set. Great communication...,5.0,"February 23, 2021",1.0,11.0,Gotland
99997,RBVTUGVC8B4LA,Love our outdoor sofa!,I had been looking for an outdoor sofa for qui...,5.0,"August 18, 2020",0.0,11.0,Gotland
99998,RUURQ2WAM7RH,Very strong rubber odor,"Very strong rubber odor, I have left the knock...",1.0,"August 28, 2019",1.0,8.0,Apexstone
99999,R21IHDS0F74O90,Product won’t last long. Already falling apart,I have not had this very long at all and alrea...,1.0,"August 21, 2020",1.0,8.0,Apexstone


In [ ]:
df = df.drop_duplicates(subset=['review_header', 'review_text'])

In [ ]:
df.shape

(83927, 8)

In [ ]:
df = df.dropna(subset=['review_header', 'review_text'])

In [ ]:
df.shape

(83926, 8)

In [ ]:
df.describe(include='all')

,review_id,review_header,review_text,rating,review_posted_date,is_verified,reviews_count,brand
count,83926,83926,83926,83926.000000,83926,83926.000000,83926.000000,83849
unique,83926,65645,83010,NaN,2989,NaN,NaN,173
top,R4I8PJ9FR0GF3,Four Stars,,NaN,"June 29, 2021",NaN,NaN,Wildone
freq,1,354,49,NaN,162,NaN,NaN,2865
mean,NaN,NaN,NaN,3.393478,NaN,0.968699,153.167493,NaN
std,NaN,NaN,NaN,1.495760,NaN,0.174132,540.320555,NaN
min,NaN,NaN,NaN,1.000000,NaN,0.000000,1.000000,NaN
25%,NaN,NaN,NaN,2.000000,NaN,1.000000,15.000000,NaN
50%,NaN,NaN,NaN,4.000000,NaN,1.000000,34.000000,NaN
75%,NaN,NaN,NaN,5.000000,NaN,1.000000,95.000000,NaN


In [ ]:
df["brand"].unique()

array(["Chef's Path", 'fullstar', 'Fullstar', 'OXO', 'Utopia Home',
       'Vtopmart', 'Wildone', 'NATURELO', 'Ultrean', 'Amazon Renewed',
       'Englander', 'Ensenior', 'ComfiLife', 'Live Conscious',
       'Ebanel Laboratories', 'Wisteria Lane', 'tobeDRI',
       'Totally Kitchen', 'Hidrate Spark', 'Coffee Gator', 'TookMag',
       'PhysioNatural', 'House Ur Home', 'ORDORA', 'Ola Prima',
       'TheraGun', 'InteVision', 'oaskys', 'SpriteGru', "Rockin' Green",
       'Everlasting Comfort', 'Incredible Bulk', 'Utopia Kitchen',
       'PECULA', 'BUG BITE THING', 'Homey Pet', "will's", 'EdenProducts',
       'DAYBETTER', '', 'Nieifi', 'Biling', 'Scriveiner',
       'Circadian Optics', 'Cotton Paradise', 'ONNAIS', 'Monument Grills',
       'CrystalTowels', 'Weigh Gram', 'SMARTRO', '500Labels', '1 Body',
       'COVAX', 'Pupford', 'ODesign', 'Sifely', 'MERRYBOX', 'KAC',
       'Annova', 'GOOD LIFE USA', 'Good Life USA', 'POWEROWL',
       'Home Intuition', 'SODUKU', 'Homestead Choice', 'B

In [ ]:
df['brand'].value_counts()

Wildone               2865
Utopia Kitchen        2347
OXO                   2250
SIGNORA WARE          2194
Vtopmart              2013
                      ... 
Fullstar                38
BLACKSMITH FAMILY       30
Vriksasana Posture      10
elv                      8
Gaggle                   3
Name: brand, Length: 173, dtype: int64

In [ ]:
df.loc[df["brand"] == "Wildone"].head()

,review_id,review_header,review_text,rating,review_posted_date,is_verified,reviews_count,brand
19270,R7IAH0U91M82G,"BUYER BEWARE, Lousy warrantee",I ordered a set and a pal bought another on my...,1.0,"September 30, 2019",1.0,59.0,Wildone
19271,R1IMW99IFV74YQ,Utter Rubbish! Gets scratched from anything,It is utter rubbish! Yes lightweight and has a...,1.0,"June 4, 2020",1.0,59.0,Wildone
19272,R1T9K23E13NNHF,"finish flaked soon after purchase, but not soo...",the finish began flaking off one of the bowls ...,1.0,"December 9, 2019",1.0,59.0,Wildone
19273,R150V76LC7LBN7,Lids cant's close plus it get rusted very easily,Lids cant close I had it replaced and still ca...,1.0,"September 15, 2019",1.0,59.0,Wildone
19274,R26Y60YDQYY7K9,"Loved, but starting to rust in less than 2 month",Loved these bowls but unfortunately they are s...,1.0,"July 11, 2020",1.0,59.0,Wildone


Mounted at /content/drive


## Generating Embeddings

In [ ]:
df['combined'] = "Title: " + df.review_header.str.strip() + "; Content: " + df.review_text.str.strip()

### Sentence Transformers

https://www.sbert.net/examples/applications/computing-embeddings/README.html

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
STE_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
STE_embeddings = STE_model.encode(df.combined.values, convert_to_tensor=True)

In [ ]:
for sentence, embedding in zip(df.combined.values, STE_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")
    break

In [ ]:
df["STE_embeddings"] = ""

In [ ]:
for index, embedding in enumerate(STE_embeddings):
    df["STE_embeddings"].iloc[index] = embedding
    print(index, len(embedding))

Streaming output truncated to the last 5000 lines.
78926 384
78927 384
78928 384
78929 384
78930 384
78931 384
78932 384
78933 384
78934 384
78935 384
78936 384
78937 384
78938 384
78939 384
78940 384
78941 384
78942 384
78943 384
78944 384
78945 384
78946 384
78947 384
78948 384
78949 384
78950 384
78951 384
78952 384
78953 384
78954 384
78955 384
78956 384
78957 384
78958 384
78959 384
78960 384
78961 384
78962 384
78963 384
78964 384
78965 384
78966 384
78967 384
78968 384
78969 384
78970 384
78971 384
78972 384
78973 384
78974 384
78975 384
78976 384
78977 384
78978 384
78979 384
78980 384
78981 384
78982 384
78983 384
78984 384
78985 384
78986 384
78987 384
78988 384
78989 384
78990 384
78991 384
78992 384
78993 384
78994 384
78995 384
78996 384
78997 384
78998 384
78999 384
79000 384
79001 384
79002 384
79003 384
79004 384
79005 384
79006 384
79007 384
79008 384
79009 384
79010 384
79011 384
79012 384
79013 384
79014 384
79015 384
79016 384
79017 384
79018 384
79019 384
79020 384

In [ ]:
df.to_csv("/content/drive/MyDrive/Product Reviews/Dataset/chunk_1_STE_embeddings.csv", index=None)

# About Clustering

https://www.sbert.net/examples/applications/clustering/README.html?

# Mamatha - KMeans Analysis

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
df["brand"].value_counts()

Wildone               2865
Utopia Kitchen        2347
OXO                   2250
SIGNORA WARE          2194
Vtopmart              2013
                      ... 
Fullstar                38
BLACKSMITH FAMILY       30
Vriksasana Posture      10
elv                      8
Gaggle                   3
Name: brand, Length: 173, dtype: int64

In [ ]:
wildone_df = df.loc[df["brand"] == "Wildone"]

In [ ]:
wildone_df.head()

,review_id,review_header,review_text,rating,review_posted_date,is_verified,reviews_count,brand,combined,STE_embeddings
19270,R7IAH0U91M82G,"BUYER BEWARE, Lousy warrantee",I ordered a set and a pal bought another on my...,1.0,"September 30, 2019",1.0,59.0,Wildone,"Title: BUYER BEWARE, Lousy warrantee; Content:...","[-0.05262017, -0.024984602, 0.105326906, -0.03..."
19271,R1IMW99IFV74YQ,Utter Rubbish! Gets scratched from anything,It is utter rubbish! Yes lightweight and has a...,1.0,"June 4, 2020",1.0,59.0,Wildone,Title: Utter Rubbish! Gets scratched from anyt...,"[-0.047085635, 0.04144992, 0.08741904, 0.04353..."
19272,R1T9K23E13NNHF,"finish flaked soon after purchase, but not soo...",the finish began flaking off one of the bowls ...,1.0,"December 9, 2019",1.0,59.0,Wildone,"Title: finish flaked soon after purchase, but ...","[0.08833083, 0.014959748, 0.09082712, 0.070817..."
19273,R150V76LC7LBN7,Lids cant's close plus it get rusted very easily,Lids cant close I had it replaced and still ca...,1.0,"September 15, 2019",1.0,59.0,Wildone,Title: Lids cant's close plus it get rusted ve...,"[-0.0014496417, 0.00086218404, -0.036945432, 0..."
19274,R26Y60YDQYY7K9,"Loved, but starting to rust in less than 2 month",Loved these bowls but unfortunately they are s...,1.0,"July 11, 2020",1.0,59.0,Wildone,"Title: Loved, but starting to rust in less tha...","[-0.01741506, -0.0073188716, 0.077372305, 0.07..."


In [ ]:
wildone_df["STE_embeddings"] = wildone_df.STE_embeddings.apply(np.array)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# Perform kmean clustering
num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(wildone_df["STE_embeddings"].to_list())
cluster_assignment = clustering_model.labels_

In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(wildone_df.combined.values[sentence_id])

In [ ]:
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  1
['Title: sizes mismarked and correct size/marking really important for a baker!; Content: I bake a lot. and to have sizes mismarked can be crucial to the finished product. Much more crucial in baking than in cooking.', 'Title: Warped on first use!; Content: The first time we ever used one of these pans, it warped terribly in the oven. It had been in the preheated oven for 5 minutes when we heard a loud bang and saw that it had warped. The pans seem extremely sturdy out of the box and I was excited to use them but sadly the quality is just not there.', 'Title: Fraud, selling a lower quality product that does not match the pictures. I\'m very unhappy.; Content: I ordered these specifically due to the heavy duty rack, shown with a thick outer wire and integrated feet in multiple images. The racks that arrived are super cheap and not only are of a different design, but have no larger gauge wire around the outside edge of the rack, and are completely made from much thinner wire t

# Jude - Agglomerative Analysis

In [75]:
from sklearn.cluster import AgglomerativeClustering

In [76]:
df["brand"].value_counts()

Wildone               2865
Utopia Kitchen        2347
OXO                   2250
SIGNORA WARE          2194
Vtopmart              2013
                      ... 
Fullstar                38
BLACKSMITH FAMILY       30
Vriksasana Posture      10
elv                      8
Gaggle                   3
Name: brand, Length: 173, dtype: int64

In [77]:
wildone_df = df.loc[df["brand"] == "Wildone"]

In [78]:
wildone_df["STE_embeddings"] = wildone_df.STE_embeddings.apply(np.array)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
corpus_embeddings = wildone_df["STE_embeddings"].to_list() /  np.linalg.norm(wildone_df["STE_embeddings"].to_list(), axis=1, keepdims=True)

In [80]:
# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

In [81]:
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(wildone_df.combined.iloc[sentence_id])

In [82]:
for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  14
['Title: BUYER BEWARE, Lousy warrantee; Content: I ordered a set and a pal bought another on my enthusiasm. BUT the .63 QT bowl developed two cracks after a month. Amazon say we are outside the 30 day return date and the supplier refers back to Amazon. So we have a bowl with a Dangerously SHARP EDGE and no one wants to step up to support the product. BUYER BEWARE', 'Title: Missing pieces; Content: My set came missing a bowl and a grater attachment. A little disappointed considering how long I waited and the fact that it’s the second largest missing which I would have used a lot more than the tiny ones', 'Title: The large bowl seems flimsy, but the rest of the set is good.; Content: I’ve only used the large bowl a handful of times, but while shredding cheese with the cut-out lid feature, the bowl split. I was disappointed as I thought this was a quality set. I’m very happy with the rest of the set! I’m not sure how to initiate a return/replacement.', 'Title: Do not like the 

# Amir - Fast Clustering Analysis

In [91]:
from sentence_transformers import SentenceTransformer, util
import torch

In [101]:
df.head()

,review_id,review_header,review_text,rating,review_posted_date,is_verified,reviews_count,brand,combined,STE_embeddings
0,R4I8PJ9FR0GF3,FALSE ADVERTISING!!!!!! NOT AS ADVERISED! ALL ...,If I could give this a ZERO RATING I would!! I...,1.0,"November 17, 2020",1.0,107.0,Chef's Path,Title: FALSE ADVERTISING!!!!!! NOT AS ADVERISE...,"[tensor(-0.0054), tensor(0.0067), tensor(0.014..."
1,R1KHEH6D35PXM,"Weird, acrid chemical smell in the lids.",I was looking forward to using these container...,1.0,"September 25, 2020",1.0,107.0,Chef's Path,"Title: Weird, acrid chemical smell in the lids...","[tensor(0.0661), tensor(-0.0475), tensor(0.053..."
2,R35MGJ0HDCABLM,Buyer Beware- NOTHING like the picture,*BUYER BEWARE * NOTHING. LIKE. THE. PICTURE. I...,1.0,"January 10, 2021",1.0,107.0,Chef's Path,Title: Buyer Beware- NOTHING like the picture;...,"[tensor(-0.0538), tensor(0.1065), tensor(0.000..."
3,RK4HRLNJWC7MT,This is what I get for being busy,I have been busy mom and not able to shop in s...,1.0,"February 13, 2021",1.0,107.0,Chef's Path,Title: This is what I get for being busy; Cont...,"[tensor(-0.0589), tensor(0.0098), tensor(0.036..."
4,RWGBX6LYL6KXK,TOO SMALL!!,I’m not quite sure why I thought these contain...,1.0,"November 16, 2020",1.0,107.0,Chef's Path,Title: TOO SMALL!!; Content: I’m not quite sur...,"[tensor(0.0348), tensor(0.0388), tensor(0.0100..."


In [84]:
df["brand"].value_counts()

Wildone               2865
Utopia Kitchen        2347
OXO                   2250
SIGNORA WARE          2194
Vtopmart              2013
                      ... 
Fullstar                38
BLACKSMITH FAMILY       30
Vriksasana Posture      10
elv                      8
Gaggle                   3
Name: brand, Length: 173, dtype: int64

In [88]:
wildone_df = df.loc[df["brand"] == "Wildone"]

In [95]:
wildone_df["STE_embeddings"] = wildone_df.STE_embeddings.apply(np.array)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
pt_tensor_from_list = torch.FloatTensor(wildone_df.STE_embeddings.to_list())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [99]:
clusters = util.community_detection(pt_tensor_from_list, min_community_size=25, threshold=0.75)

In [100]:
#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", wildone_df.combined.iloc[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", wildone_df.combined.iloc[sentence_id])


Cluster 1, #109 Elements 
	 Title: lids are useless; Content: Lids are utterly useless, do not buy. There are way cheaper options for just the bowls. Lids do not fit!!!
	 Title: Lids do not fit properly; Content: I was very excited about this purchase, the bowls stack well and the lids with accessories seem great. But, unfortunately, the lids do not fit properly, they will not seal therefore making them useless.
	 Title: Lids do not seal properly; Content: These bowls are great, specially the bottom rubber around the bowl is so good when mixing. The main problem that i have is the Lids, these do not seal properly even after pressing all the way down from all side these lids keep coming off. only two our of five close properly (stay in place) the other seems to be the wrong size as it just pop right off. If these LIDS get fix i will five these mixing bowls 5 starts, but not being able to seal the mix create more of a mess because i have to use something else to stored what i just mixed